# DotDict class

What I want it to do:
- works with any dictionary whose keys are strings (and whose nested dictionaries keys are strings as well)
- `dotdict_test = DotDict(dict_test)` gives a DotDict object, `dict_test = dict(dotdict_test)` gives back the dictionary
- can access the entries through the "dot" notation AND the regular dictionary index
- setting / accessing / deleting an attribute sets / accesses / deletes the same entry in the dictionary
- can create an empty dotdict with `DotDict()`

initialise:
- `dotdict_test = DotDict(my_dict)`
- `empty_dotdict = DotDict()`

retrieve the dictionary: `my_dict = dict(dotdict_test)`

JSON-like print: `print(dotdict_test)`

`DotDict(eval(str(dotdict_test)))` returns the same DotDict object for dictionaries whose types themselves work with eval

look for key/attribute:
- `hasattr(dotdict.a.b, c)`
- `"c in dotdict.a.b"`
- `"c" in dotdict.a.b.keys()`
- `"c in list(dotdict.a.b)"`
- `"c" in dir(dotdict.a.b)` (`dir` will list `dict` and `DotDict` methods as well)

get:
- `dotdict.a.b.c`
- `dotdict["a"]["b"]["c"]`
- `dotdict.a["b"].c`
- `getattr(dotdict.a.b, c)`

set:
- `dotdict.a.b.c = 0`

delete:
- `dotdict.clear()` empties the DotDict object

TODO:
- setting items is not recursive => OK
- make it work with any type of mapping, not just dicts ? => needs to work recursively => OK
- test deleting
- add a copy method ? => return a shallow copy of DotDict ? => OK
- add pretty prints when called by `print` => OK
- check that every method of `dict` is working as expected
- create new nice template for docstrings

In [1]:
# import sys
# sys.setrecursionlimit(100)

# def func():
#     func()

# func()

In [2]:
# import IPython

# IPython.Application.instance().kernel.do_shutdown(True)
# dir(IPython.Application.instance().kernel)

```python
(collections.abc.ValuesView,
 collections.abc.MappingView,
 collections.abc.Collection,
 collections.abc.Sized,
 collections.abc.Iterable,
 collections.abc.Container,
 object)
```

In [3]:
# %reset -f
# %load_ext autoreload
# %autoreload 2

from pathlib import Path
import sys
from types import MappingProxyType
from collections.abc import ItemsView, ValuesView, Container, Iterator, Mapping, Mapping
# ValuesView parent classes
from collections.abc import MappingView, Collection, Sized, Iterable

# ROOT_DIR_PATH = str(Path(__file__).resolve().parent)
ROOT_DIR_PATH = str(Path.cwd().resolve().parent)
if ROOT_DIR_PATH not in sys.path:
    sys.path.insert(0, ROOT_DIR_PATH)

from dotdict import DotDict, DotDictValues

# to prevent the kernel from dying when testing
sys.setrecursionlimit(1000)

## basic tests

In [4]:
# empty DotDict, basic insertion
test = DotDict(_verbose=True)
print(test)
test.lol = 3
test["haha"] = 6
print(test)

__init__ call
__str__ call
{}
__setitem__ call
__setitem__ call
__str__ call
{'lol': 3, 'haha': 6}


In [5]:
# standard dict
test_dict = {'a': {'foo': 'bar'}, 'b': {'lol': 'kekw', 'hehe': 'haha'}, 'c': {}}
test = DotDict(test_dict, _verbose=True)
print(test)

__init__ call
__str__ call
{'a': {'foo': 'bar'}, 'b': {'lol': 'kekw', 'hehe': 'haha'}, 'c': {}}


In [6]:
# vars, dict as MappingProxyType objects
vars(test)

__dict__ call
_view call
__getitem__ call
__init__ call
__getitem__ call
__init__ call
__getitem__ call
__init__ call
__repr__ call
__repr__ call
__repr__ call


mappingproxy({'a': DotDict({'foo': 'bar'}),
              'b': DotDict({'lol': 'kekw', 'hehe': 'haha'}),
              'c': DotDict({})})

In [7]:
test.__dict__

__dict__ call
_view call
__getitem__ call
__init__ call
__getitem__ call
__init__ call
__getitem__ call
__init__ call
__repr__ call
__repr__ call
__repr__ call


mappingproxy({'a': DotDict({'foo': 'bar'}),
              'b': DotDict({'lol': 'kekw', 'hehe': 'haha'}),
              'c': DotDict({})})

In [8]:
# _root and _path_to_root accessible but not protected from rewrites
print(test._root)
print(test._path_to_root)
print(test._view)
print(test._verbose)

None
None
_view call
__getitem__ call
__init__ call
__getitem__ call
__init__ call
__getitem__ call
__init__ call
__repr__ call
__repr__ call
__repr__ call
{'a': DotDict({'foo': 'bar'}), 'b': DotDict({'lol': 'kekw', 'hehe': 'haha'}), 'c': DotDict({})}
True


In [9]:
# get the dict back (does not call __dict__)
print(dict(test))

{'a': {'foo': 'bar'}, 'b': {'lol': 'kekw', 'hehe': 'haha'}, 'c': {}}


In [10]:
# test._root = 2
# test["_path_to_root"] = "prout"

In [11]:
# __iter__
for item in iter(test):
    print(item)

a
b
c


In [12]:
# __reversed__
for item in reversed(test):
    print(item)

c
b
a


In [13]:
# DotDict of DotDict
dotdict_output = DotDict(test)
dotdict_output._verbose
# print(dotdict_output)
# print(dotdict_output._root)

False


False

In [ ]:
# weird DotDict instanciation, with accepted hidden variables
test = DotDict(blbl="lol", stuff=74, null_value=None, _verbose=True)
print(test)
print(test.null_value)
# print(vars(test))

In [ ]:
# another weird DotDict instanciation
test = DotDict({
    "lol": [["foo", "bar"], [1,2]],
    "haha": "abc"
})
print(test)

In [ ]:
# dict(dotdict_object) returns the dictionary
dict_test_returned = dict(test)
print(type(dict_test_returned["lol"]))
print(dict_test_returned)

In [ ]:
# eval(str(dotdict_object)) returns the dictionary
dict_test_returned = eval(str(test))
print(dict_test_returned)
print(type(dict_test_returned))

In [ ]:
# __eq__
test = {"a": 1}
print(test == DotDict(test, _verbose=True))
print(DotDict(test, _verbose=True) == test)

In [ ]:
# __doc__
print(DotDict.__doc__)

In [ ]:
# help(DotDict)

In [ ]:
help(Path)
# Path.__init__

In [ ]:
test_path_str = "&@$£"
Path(test_path_str)

## init test DotDict objects

In [ ]:
dict_test = {
    "param_1": "great",
    "param2": 2,
    # "_lol": "lol",
    # "items": ["a", "b", "c"],
    "param3": {
        "subparam": -5.2,
        # "2": "lol",
        # 1: "haha"
    },
    "param4": [6,1,2],
    "lol": "haha"
}
dotdict_test_1 = DotDict(dict_test, _verbose=True)
print(dotdict_test_1)

In [ ]:
print(repr(dotdict_test_1))

In [ ]:
# goodness me
eval(repr(dotdict_test_1)) == dotdict_test_1

In [ ]:
eval(repr(dotdict_test_1)) == test

In [ ]:
# replace mappings with dicts when added
dotdict_1 = DotDict({
    "a": 0,
    "b": 1,
    "c": 2
})
dotdict_2 = DotDict({
    "a": 0,
    "b": 1,
    "c": 2
})
dotdict_3 = MappingProxyType({
    "a": {
        "name": "lol",
        "path": "./"
    },
    "b": {
        "name": "foo",
        "path": "./bar/"
    },
    "c": 2
})
dict_dotdict = {
    "first": dotdict_1,
    "second": dotdict_2,
    "third": dotdict_3
}
dotdict_test_2 = DotDict(dict_dotdict)
print(dotdict_test_2)
print(type(dotdict_test_2.third))

## get values

In [ ]:
# accessing values
print(dotdict_test_1.param3)
print(type(dotdict_test_1.param3))
print(type(dotdict_test_1["param3"]))
print(dotdict_test_1["param4"])
print(hasattr(dotdict_test_1, "param1"))
print(hasattr(dotdict_test_1, "fdfd"))
print("param2" in dotdict_test_1.keys())
print("subparam" in dotdict_test_1.param3.keys())
print("subparam" in dir(dotdict_test_1.param3))
print(dotdict_test_1.param3.subparam)
print(getattr(dotdict_test_1.param3, "subparam"))

## set values

In [ ]:
# setting values
dotdict_test_1.param5 = {1,2,3}
dotdict_test_1["param6"] = "blblblb"
dotdict_test_1["param7"] = {
    "lol": {
        "Lol0": 0,
        # "1": 2,
        # "_lol3": 6,
        # "_ABC": 4
        # 2: "lol"
    },
    "haha": "prout"
}
print(dotdict_test_1)
print("#"*50)
setattr(dotdict_test_1, "param8", "toto")
# dotdict_test["246"] = 4
dotdict_test_1.param4[0] = -5
dotdict_test_1.param7.haha = "awesome"
print(dotdict_test_1.param7.haha)
print("#"*50)
sub_dotdict = dotdict_test_1.param7
print(sub_dotdict)
print("#"*50)
print(type(sub_dotdict))
print("#"*50)
# print(sub_dotdict._root)
# print(sub_dotdict._path_to_root)
sub_dotdict.foo = "bar"
print(dotdict_test_1)
print("#"*50)
print(sub_dotdict)

## delete values

In [ ]:
# delete values
delattr(dotdict_test_1, "param5")
del dotdict_test_1["param7"]
del dotdict_test_1.param6
print(dotdict_test_1)

In [ ]:
# delete more items
del dotdict_test_2.third.b
dotdict_test_2

In [ ]:
sub_dotdict = dotdict_test_2.third
del sub_dotdict.a.path
print(dotdict_test_2)
print(sub_dotdict)

## \_\_str__ and \_\_repr__

`dict.\_\_str__` and `dict.\_\_repr__` return the same string

=> ~~maybe delete `DotDict.\_\_str__` ?~~ 

=> make `DotDict.\_\_str__` return `dict.\_\_str__` and `DotDict.\_\_repr__` the same string enclosed with `DotDict(...)`

In [ ]:
dotdict_test = DotDict({
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout",
        "e": {
            "foo": "bar"
        }
    }
}, _verbose=True)

In [ ]:
print(dotdict_test)

In [ ]:
print(repr(dotdict_test))

In [ ]:
dict.__str__({"1": 2})

In [ ]:
dict.__repr__({"1": 2})

In [ ]:
object.__str__({"1": 2})

In [ ]:
object.__repr__({"1": 2})

In [ ]:
object.__repr__(dotdict_test)

In [ ]:
object.__str__(dotdict_test)

## \_\_dict__

In [ ]:
dotdict_test = DotDict({
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout",
        "e": {
            "foo": "bar"
        }
    }
}, _verbose=True)
print(dotdict_test)

## test dict methods

### copy

#### dict behavior

In [ ]:
# dict  behavior
dict_test = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout",
        "e": {
            "foo": "bar"
        }
    }
}

In [ ]:
# make a copy of a nested element containing nested elements
d_copy = dict_test["d"].copy()
print(dict_test)
print(d_copy)

In [ ]:
# modifying the copy nested elements modifies the original nested elements
d_copy["e"]["new"] = "value"
print(d_copy)
print(dict_test)

In [ ]:
# vice versa
dict_test["d"]["e"]["newnew"] = "VALUE"
print(d_copy)
print(dict_test)

#### DotDict behavior

In [ ]:
dotdict_test = DotDict({
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout",
        "e": {
            "foo": "bar"
        }
    }
}, _verbose=True)

In [ ]:
# shallow copy test
dotdict_test_copy = dotdict_test.copy()
# this doesn't change the copy
dotdict_test.a = "fgfgf"
# this does
dotdict_test.d["lol"] = 10
dotdict_test.d.foo = "bar"
print(dotdict_test)
print()
print(dotdict_test_copy)
print()
print(type(dotdict_test_copy))

In [ ]:
dotdict_test._verbose

In [ ]:
dotdict_test_copy._verbose

In [ ]:
# nested copy test
# make a copy of a nested element containing nested elements
dotdict_test_copy = dotdict_test["d"].copy()
print(dotdict_test)
print(dotdict_test_copy)
print(type(dotdict_test_copy))
print(dotdict_test_copy._verbose)
print(dotdict_test._verbose)

In [ ]:
type(dotdict_test["d"])

In [ ]:
dotdict_test["d"].copy?

In [ ]:
print(dotdict_test["d"].copy)

In [ ]:
test = dotdict_test["d"]
test.copy?

In [ ]:
{"a": {}}["a"].copy

In [ ]:
dotdict_test.d.copy?

In [ ]:
# modifying the copy nested elements modifies the original nested elements
dotdict_test_copy["e"]["new"] = "not value"
print(dotdict_test_copy)
print(dotdict_test)

In [ ]:
# vice versa
dotdict_test["d"]["e"]["newnew"] = "NOT VALUE"
print(dotdict_test_copy)
print(dotdict_test)

is DotDict(dict(dotdict_object)) a shallow copy of dotdict_obj ?

=> is DotDict(dotdict_object) a shallow copy of dotdict_obj ?

=> is dict(dict_object) a shallow copy of dict_object ?

In [ ]:
dict_test = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout",
        "e": {
            "foo": "bar"
        }
    }
}
dotdict_test = DotDict(dict_test, _verbose=True)

In [ ]:
# dict behavior
# dict_test.copy() is a shallow copy of dict_test
dict_test_copy_1 = dict_test.copy()
print(dict_test_copy_1 is dict_test)
print(dict_test_copy_1["d"] is dict_test["d"])
print()

# dict(dict_test) is also a shallow copy of dict_test
dict_test_copy_2 = dict(dict_test)
print(dict_test_copy_2 is dict_test)
print(dict_test_copy_2["d"] is dict_test["d"])
print()

In [ ]:
# dotdict behavior
# dotdict_test_copy_1 = dotdict_test.copy()
# dotdict_test_copy_1 = DotDict(
#     dotdict_test,
#     _check=False,
#     _root=dotdict_test._root,
#     _path_to_root=dotdict_test._path_to_root,
#     _verbose=dotdict_test._verbose
# )
dotdict_test_copy_1 = DotDict(dotdict_test)
print(dotdict_test_copy_1 is dotdict_test)
print()
print(dotdict_test_copy_1["d"] is dotdict_test["d"])
print()
...

In [ ]:
# print(dict.__doc__)

#### references

In [ ]:
# now THAT is crazy
print(dotdict_test["a"] is dotdict_test["a"])
print(dotdict_test["d"] is dotdict_test["d"])
# => create a "referenced_dotdicts" dictionary to return the same DotDict everytime ?
#     => update DotDict.__new__ to only send a new object if it was never accessed before
#     => udpate DotDict.__init__ to reference each of its nested DotDict once (will create a copy of each DotDict in memory)
#     => clear key/values with "del" to clear memory when they're not referenced anymore
# => maybe use functools.cache ?

In [ ]:
DotDict(dotdict_test)

In [ ]:
dotdict_test

### vars

In [ ]:
# vars
vars(dotdict_test_1)

In [ ]:
print(dotdict_test_1)

### update

In [ ]:
# update test
...

In [ ]:
# dotdict_test_1.param3.update({"subparam2": "foo"})
# dotdict_test_1.param3

### iter

In [ ]:
# iter
for key in iter(dotdict_test_1):
    print(key)

### clear

In [ ]:
# clear
dotdict_test = DotDict({
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
})
dotdict_test.clear()
dotdict_test

In [ ]:
dotdict_test = DotDict({
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
})
dotdict_test["d"].clear()
dotdict_test

In [ ]:
dict_test = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
}
dict_test["d"].clear()
dict_test

In [ ]:
dotdict_test = DotDict({
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
})
place_holder = dotdict_test["d"]
place_holder.clear()
dotdict_test

In [ ]:
dict_test = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
}
place_holder = dict_test["d"]
place_holder.clear()
dict_test

### get

In [ ]:
# get
dict_test = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
}
type(dict_test.get("d"))

In [ ]:
print(dict_test.get("doesnt_exist"))

In [ ]:
dotdict_test = DotDict({
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
}, _verbose=True)
type(dotdict_test.get("d"))

In [ ]:
print(dotdict_test.d.get("kekw"))

In [ ]:
print(dotdict_test.get("doesnt_exist"))

### _view

In [ ]:
view_result = dotdict_test._view
print(type(view_result))
for key in view_result.keys():
    print(type(view_result[key]))

In [ ]:
print(view_result["d"]._root)
print(view_result["d"]._path_to_root)

### values

#### dict behavior

In [ ]:
# dict behavior
dict_test = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
}

In [ ]:
dict_values = dict_test.values()
dict_values

In [ ]:
dict_test["a"] = 2
dict_test

In [ ]:
# __str__
print(dict_values)

In [ ]:
# __repr__
print(repr(dict_values))

In [ ]:
# non subscriptable
# dict_values[0]

In [ ]:
isinstance(dict_values, ValuesView)

In [ ]:
# __contains__
2 in dict_values

In [ ]:
# __iter__
for value in dict_values:
    print(value)

In [ ]:
dict_values_type = type(dict_values)
dict_values_type

In [ ]:
mapping = dict_values.mapping
mapping

In [ ]:
list(dict_values)

In [ ]:
iter(dict_values)

In [ ]:
# even the mapping output is dynamic !
dict_test["c"] = 10
mapping

In [ ]:
dict_values

In [ ]:
# __or__ doesn't work
# dict_values | dict_values

In [ ]:
# dict_values_type.__new__(dict_values_type, {})

In [ ]:
# dict_values.__new__(dict_values_type, {})

In [ ]:
# object.__new__(dict_values_type, {})

In [ ]:
# dir(dict_values)

In [ ]:
# type(dict_values).__mro__

In [ ]:
# ValuesView.__mro__

In [ ]:
# dict_values.haha = 5

#### dotdict behavior

In [ ]:
# dotdict behavior
dict_test = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
}
dotdict_test = DotDict(dict_test, _verbose=True)
print(dotdict_test)

#### DotDictValues type

abc.collections ValuesView inheritance:
- MappingView
    - Sized
- Collection
    - Iterable
    - Container

abc.collections ValuesView requirements:
- \_\_contains__, \_\_iter__, \_\_len__

In [ ]:
# new DotDictValues object
# dotdict_values = DotDictValues.__new__(DotDictValues, dotdict_test)
dotdict_values = dotdict_test.values()

In [ ]:
# __init__
# type(dotdict_values)(dotdict_test)

In [ ]:
# __str__
print(dotdict_values)

In [ ]:
# __repr__
print(repr(dotdict_values))

In [ ]:
# __len__
print(len(dotdict_values))

In [ ]:
# values list updated on DotDict update
dotdict_test["a"] = 2
print(dotdict_test)

In [ ]:
print(dotdict_values)

In [ ]:
# non subscriptable
# dotdict_values[0]

In [ ]:
# __iter__
for value in dotdict_values:
    print(value)
    print(type(value))

In [ ]:
# can be made into a list
print(list(dotdict_values))
print()
# same thing but without a DotDictValues.__len__ call
print(list(iter(dotdict_values)))

In [ ]:
# __reversed__
reversed_values_iterator = reversed(dotdict_values)
for value in reversed_values_iterator:
    print(value)

In [ ]:
# __contains__
print(2 in dotdict_values)
print()
print(DotDict({'lol': 'haha','kekw': 'prout'}) in dotdict_values)
print()
print({'lol': 'haha','kekw': 'prout'} in dotdict_values)
print()
print("lul" in dotdict_values)

In [ ]:
# mapping
# MappingProxyType.__str__ calls the original object __str__ method (here, DotDict)
# same for MappingProxyType.__repr__
dotdict_values_mapping = dotdict_values.mapping
str(dotdict_values_mapping)
print(dotdict_values_mapping)
print(type(dotdict_values_mapping))
print(repr(dotdict_values_mapping))
# changes to the values are reflected in dotdict_values_mapping
dotdict_test["a"] = 20
print(dotdict_values)
print(dotdict_values_mapping)
# unmutable
# dotdict_values_mapping['a'] = 2

In [ ]:
isinstance(dotdict_values, ValuesView)

In [ ]:
# access to attributes denied
# get
# dotdict_values.b
# set
# dotdict_values.b = 7
# delete
# del dotdict_values.b

In [ ]:
# __or__ doesn't work
# dotdict_values | dotdict_values

In [ ]:
print(dotdict_values._dotdict_hook)

In [ ]:
issubclass(DotDictValues, object)

In [ ]:
issubclass(DotDictValues, Container)

In [ ]:
issubclass(DotDictValues, Iterable)

In [ ]:
issubclass(DotDictValues, Sized)

In [ ]:
issubclass(DotDictValues, Collection)

In [ ]:
issubclass(DotDictValues, MappingView)

In [ ]:
issubclass(DotDictValues, ValuesView)

In [ ]:
DotDictValues.__mro__

In [ ]:
issubclass(DotDict, Mapping)

In [ ]:
isinstance({}, Mapping)

In [ ]:
# dir(DotDictValues)

In [ ]:
# dir(ValuesView)

In [ ]:
type(dict_values)

In [ ]:
# differences between __dir__:

print("collections.abc.ValuesView VS dict_values")
_ = [print(" "*4 + item) for item in dir(ValuesView) if not (item in dir(dict_values))]
print()
_ = [print(" "*4 + item) for item in dir(dict_values) if not (item in dir(ValuesView))]
print()

print("DotDictValues VS collections.abc.ValuesView")
_ = [print(" "*4 + item) for item in dir(DotDictValues) if not (item in dir(ValuesView))]
print()
_ = [print(" "*4 + item) for item in dir(ValuesView) if not (item in dir(DotDictValues))]
print()

print("DotDictValues VS dict_values")
_ = [print(" "*4 + item) for item in dir(DotDictValues) if not (item in dir(dict_values))]
print()
_ = [print(" "*4 + item) for item in dir(dict_values) if not (item in dir(DotDictValues))]
print()

print("dotdict_values VS DotDictValues")
_ = [print(" "*4 + item) for item in dir(dotdict_values) if not (item in dir(DotDictValues))]
print()
_ = [print(" "*4 + item) for item in dir(DotDictValues) if not (item in dir(dotdict_values))]
print()

In [ ]:
type(iter(dict_test))

In [ ]:
_ = [print(item) for item in dir(dict_values) if not (item in dir(ValuesView))]

In [ ]:
# dict_values.l = 2

In [ ]:
_ = [print(item) for item in dir(type(dict_values)) if not (item in dir(dict_values))]

In [ ]:
dotdict_values.l = 2

In [ ]:
dir(dotdict_values)

In [ ]:
dir(DotDictValues)

### items

In [ ]:
dict_test = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": {
        "lol": "haha",
        "kekw": "prout"
    }
}
type(dict_test.items())

In [ ]:
dict_test.items()[0]

In [ ]:
len(dict_test.items())

In [ ]:
for item in dict_test.items():
    print(item)
print(type(item))

In [ ]:
# dir(type(dict_test.items()))

In [ ]:
# dir(DotDict)

In [ ]:
# _ = [print(item) for item in dir(DotDict) if item not in dir(type(dict_test.items()))]

In [ ]:
# _ = [print(item) for item in dir(type(dict_test.items())) if item not in dir(DotDict)]

In [ ]:
# dir(dict)

In [ ]:
# _ = [print(item) for item in dir(list) if item not in dir(type(dict_test.items()))]

In [ ]:
# _ = [print(item) for item in dir(type(dict_test.items())) if item not in dir(list)]

In [ ]:
dict_items = dict_test.items()
dict_items

In [ ]:
# DotDictItems class
# dir(DotDict_items)

In [ ]:
DotDict_items([("a", 0)])

In [ ]:
type(dict.items({}))

In [ ]:
type(dict_test.keys())

In [ ]:
dict_test.keys().mapping

In [ ]:
# len test
print(len(dict_test), len(dotdict_test))

In [ ]:
# __eq__ ?
place_holder = dotdict_test_1
dotdict_test_1 == place_holder

In [ ]:
dotdict_test_1 == dotdict_test_2

In [ ]:
"param_1" in dotdict_test

In [ ]:
dict.update?

In [ ]:
dotdict_test.param7.lol.Lol0

In [ ]:
dict(dotdict_test)

In [ ]:
print(type(result_dict["first"]))

In [ ]:
print(type(dotdict_test.first))

In [ ]:
dir(dict)

In [ ]:
dict.__or__?

### keys

### dict comprehension

{attr if attr.startswith("__") else attr.upper(): v for attr, v in future_class_attrs.items()}

In [ ]:
dict_test = {
    "a": 0,
    "B": 1,
    "c": 2
}
dict_comp = {(key.upper()): (value*2) for (key, value) in dict_test.items()}
dict_comp

## failure cases

In [ ]:
dotdict_test.lol

In [ ]:
dotdict_test["haha"]

In [ ]:
dotdict_test[0] = "lol"

In [ ]:
dotdict_test["0"] = "lol"

In [ ]:
del dotdict_test.lol

In [ ]:
del dotdict_test["haha"]

In [ ]:
# forbidden attribute names
dict_attribute_names = [item for item in dir(dict) if not ("_" in item)]
dict_attribute_names

# Class inheritance

`super(type[, instance_or_type])`

- `super` returns a proxy item used to call methods from parent or sibling classes
- `super(type)` returns unbound super object
- `super(type, instance)` instance must be instance of type (isinstance(instance, type))
- `super(type, type2)` type2 must be subclass of type (issubclass(type2, type))

`super` is only used when you don't want to name the parent classes by name => behavior determined at rutime using the MRO

In [ ]:
%reset -f
class B:
    def __init__(self, b_atr: int):
        print("init B")
        self.b_atr = b_atr
        
    def meth_b(self):
        print("this is a method of the class B")
        
    # def meth(self):
    #     print("this is the method 'meth' of the class A")

# a_test = A(2)
# print(a_test.a_atr)


class C:
    def __init__(self, c_atr: int):
        print("init C")
        self.c_atr = c_atr
        
    def meth_c(self):
        print("this is a method of the class C")
        
    # def meth(self):
    #     print("this is the method 'meth' of the class B")
        

class D(C):
    def __init__(self, c_atr: int, d_atr: int):
        print("init D")
        # print(super())
        # super().__init__(c_atr)
        C.__init__(self, c_atr)
        self.d_atr = d_atr
        print("end init D")
        
    def meth_d(self):
        print("this is a method of the class D")
        
# d_test = D(1,2)
# print(d_test.d_atr)
# print(d_test.c_atr)
# d_test.meth_d()
# d_test.meth_c()


class A(B, D):
    def __init__(self, a_atr: int, b_atr: int, c_atr: int, d_atr: int):
        print("init A")
        B.__init__(self, b_atr)
        D.__init__(self, c_atr, d_atr)
        self.a_atr = a_atr
        print("end init A")
        
    # def super_tests(self):
    #     print(super())
    #     print(super(self.__class__, self))
    #     # print(super(...))

        
# print("A.__class__:", A.__class__)
# print("type(A):", type(A))
# print("MRO:")
for item in A.__mro__:
    print(item)
print()

a_test = A(1,2,3,4)
print(a_test.a_atr)
print(a_test.b_atr)
print(a_test.c_atr)
print(a_test.d_atr)
a_test.meth_b()
a_test.meth_c()
a_test.meth_d()

In [ ]:
super(A, B)

In [ ]:
super?

In [ ]:
%reset -f

class Square:
    
    def __init__(self, side_length):
        self.side_length = side_length
    
    @property
    def area(self):
        return self.side_length ** 2
        
square_test = Square(3)
print(square_test.side_length)
print(square_test.area)


class Cube(Square):
    
    @property
    def area(self):
        return super().area * 6
    
    @property
    def volume(self):
        return self.side_length ** 3

cube_test = Cube(4)
print(cube_test.side_length)
print(cube_test.area)
print(cube_test.volume)

In [ ]:
%reset -f

class Tokenizer:
    """Tokenize text"""
    def __init__(self, text):
        print('Start Tokenizer init')
        self.tokens = text.split()
        print('End Tokenizer init')


class WordCounter(Tokenizer):
    """Count words in text"""
    def __init__(self, text):
        print('Start WordCounter init')
        super().__init__(text)
        self.word_count = len(self.tokens)
        print('End WordCounter init')


class Vocabulary(Tokenizer):
    """Find unique words in text"""
    def __init__(self, text):
        print('Start Vocabulary init')
        super().__init__(text)
        self.vocab = set(self.tokens)
        print('End Vocabulary init')


class TextDescriber(WordCounter, Vocabulary):
    """Describe text with multiple metrics"""
    def __init__(self, text):
        print('Start TextDescriber init')
        super().__init__(text)
        print('End TextDescriber init')
        
        
# for item in TextDescriber.__mro__:
#     print(item)


td = TextDescriber('row row row your boat')
print('--------')
print(td.tokens)
print(td.vocab)
print(td.word_count)

In [ ]:
# parent, child, grandchild:
%reset -f

class Parent:
    def method(self):
        print("parent method")

class Child(Parent):
    def method(self):
        print("child method")
        print(super())
        super().method()

class Grandchild(Child):
    def method(self):
        print("grandchild method")
        print(super())
        super().method()
    
for item in Grandchild.__mro__:
    print(item)
print()

Grandchild().method()

In [ ]:
# child and 2 parents:
%reset -f

class Parent1:
    def method(self):
        print("parent 1 method")
        
class Parent2:
    def method(self):
        print("parent 2 method")
        
class Child(Parent1, Parent2):
    def method(self):
        print("child method")
        print(super())
        print(super(Parent1, self))
        # super(Parent1, self).method()
        # super(Parent2, self).method()
        
for item in Child.__mro__:
    print(item)
print()

Child().method()

In [ ]:
# Child, Parent1, Parent2, Grandparent:
%reset -f

class Grandparent:
    def method(self):
        print("grandparent method")

class Parent1(Grandparent):
    def method(self):
        print("parent1 method")
        print(super())
        super().method()

class Parent2(Grandparent):
    def method(self):
        print("parent2 method")
        print(super())
        super().method()

class Child(Parent1, Parent2):
    def method(self):
        print("child method")
        print(super())
        # print(super().super())
        super().method()
        
for item in Child.__mro__:
    print(item)
print()

Child().method()

In [ ]:
%reset -f

class Parent1:
    def __init__(self):
        print("parent 1 init")
        
class Parent2:
    def __init__(self):
        print("parent 2 init")
        
class Child(Parent1, Parent2):
    def __init__(self):
        print("child init")
        super().__init__()

Child()

# DUMP

In [ ]:
def test(*args, **kwargs):
    print(args)
    print(kwargs)

args_test = (1,2,3)
kwargs_test = {"lol": 0, "lblb": 1}
test(*args_test, **kwargs_test)

In [ ]:
pattern = re.compile("^[a-zA-Z]\w*$")
pattern

In [ ]:
test_list = [
    "MyClass",
    "_a_hidden_parametter",
    "__dunder__",
    "param_6",
    "YikeS_23",
    "aLBERT2dse",
    "0",
    "25laurence",
    "Myclass#6",
    "This doesn't work either",
    "test's"
]
for str_ in test_list:
    if pattern.match(str_):
        print("match")
    else:
        print("no match")

In [ ]:
class Test:
    pass
test_dir_return = Test().__dir__()
test_dir_return

In [ ]:
type(test_dir_return)

In [ ]:
class NullTest:
    def __init__(self):
        self.lol = "kaka"
        print(self.lol)
        # self = None


null_test = NullTest()
null_test.__dict__

In [ ]:
DotDict({'one': 1, 'three': 3}, two=2)

In [ ]:
test = ["a", "l", "z"]
test.append("e")
test

In [ ]:
dir(object)

In [ ]:
list(vars(object).keys())

In [ ]:
len(vars(object).keys())

In [ ]:
len(dir(object))

In [ ]:
dir(dict)

In [ ]:
dir(dotdict_test)

In [ ]:
vars(dict)

In [ ]:
dict_test = {"lol": "haha"}
vars(dict_test)

In [ ]:
vars(dotdict_test)

In [ ]:
vars(DotDict)

In [ ]:
list_test = [1,2,3]
vars(list_test)

In [ ]:
class Test:
    pass

object_test = Test()
# dir(object_test)

In [ ]:
vars(object_test)

In [ ]:
object_test.b = 2
object_test.b

In [ ]:
object_test.__dict__["b"] = 3
object_test.__dict__["c"] = 18
print(object_test.b, object_test.c)

In [ ]:
vars(object_test)["d"] = -5
object_test.d

In [ ]:
# delete an attribute with vars and __dict___ ?
del vars(object_test)["d"]
del object_test.__dict__["c"]
object_test.c

In [ ]:
test = [1,2,3]
test

In [ ]:
copy_test = test.copy()
copy_test

In [ ]:
copy_test.append(4)
copy_test

In [ ]:
test

In [ ]:
copy_test = test + [4]
copy_test

In [ ]:
test

In [ ]:
class Test:
    def __setitem__(self, key, value):
        print("__setitem__ call")
        super().__setitem__(key, value)
    def __setattr__(self, key, value):
        print("__setattr__ call")
        super().__setattr__(key, value)

test = Test()
test.a = "b"
test.a

In [ ]:
MappingProxyType?

In [ ]:
test = object()
test.lol = 2
test

In [ ]:
test.__dict__ = {}

In [ ]:
test.__setattr__("__dict__", {})
test

In [ ]:
test.__setattr__

In [ ]:
dict.__str__

In [ ]:
test = {"lol": 2}
test_it = iter(test)
isinstance(test_it, Iterator)

In [ ]:
test = "lol"
test_it = iter(test)
isinstance(test_it, Iterator)

In [ ]:
test = (1,2,3)
test_it = iter(test)
isinstance(test_it, Iterator)

In [ ]:
next(test_it)

In [ ]:
[e for e in "lol"]

In [ ]:
"lol"[1]

In [ ]:
test = [1,2,3]
test_it = iter(test)
next(test_it)

In [ ]:
test = {0,1,2}
iter(test)

In [ ]:
a = 0
try:
    a = 1/0
except:
    pass
a

In [ ]:
from collections.abc import Mapping, Iterable, Sequence, MutableSequence
from typing import Any
# isinstance(DotDict(), Mapping)
isinstance("lol", MutableSequence)

In [ ]:
isinstance(vars(DotDict()), Mapping)

In [ ]:
isinstance("abc", Mapping)

In [ ]:
isinstance((1,2,3), MutableSequence)

In [ ]:
iter(range(0,10))

In [ ]:
dir(object())

In [ ]:
dir(dict())

In [ ]:
test = b"still allows embedded 'single' quotes"
iter(test)

In [ ]:
type(test) is str

In [ ]:
# iter(1845)

In [ ]:
isinstance(iter(1845), Iterator)

In [ ]:
test = bytearray(b'Hi!')
iter(test)
isinstance(iter(test), Iterator)

In [ ]:
filter?

In [ ]:
test = list(filter(None, [3,0,1,2]))
test

In [ ]:
# sum(), any(), all(), max(), min(), and len()
from functools import reduce

_len = lambda it: reduce(
    lambda value, element: value + 1,
    it,
    0
)

_sum = lambda it: reduce(
    lambda value, element: value + element,
    it
)

_any = lambda it: reduce(
    lambda value, element: bool(value) or bool(element),
    it,
    False
)

_all = lambda it: reduce(
    lambda value, element: bool(value) and bool(element),
    it,
    True
)

_max = lambda it: reduce(
    lambda value, element: element if element > value else value,
    it
)

_min = lambda it: reduce(
    lambda value, element: element if element < value else value,
    it
)

print(_len(["a", "b", "c"]))
print(_sum([3,1,2]))
print(_any([False, False]))
print(_any([False, True]))
print(_all([True, True]))
print(_all([False, True]))
print(_max([1,5,-2]))
print(_min([1,5,-2]))

In [ ]:
test_lamda = lambda x, y: x + y
test_lamda(2,3)

In [ ]:
def title_printer(title: str, fill_char: str = "#", print_: bool = True) -> str:
    """
    ###########################
    ### super awesome title ###
    ###########################
    """
    # create title string
    title_lines = []
    title_lines.append("####" + "#"*len(title) + "####")
    title_lines.append("### " + title + " ###")
    title_lines.append("####" + "#"*len(title) + "####")
    title_string = "\n".join(title_lines)
    # print it and return it
    if print_:
        print(title_string)
    return title_string

print(title_printer("super awesome title", print_=False))

In [ ]:
print(title_printer("title printer", print_=False))

In [ ]:
dir(object)

In [ ]:
isinstance([], Container)

In [ ]:
dir([])

In [ ]:
test_1 = DotDict({"a": 0})
test_2 = DotDict({"a": 0})
test_1 is test_2

In [ ]:
test_1 == test_2

In [3]:
class Test:
    def __init__(self, *args):
        if args:
            if isinstance(args[0], Test):
                print("thing is instance of Test")
test = Test()
test_2 = Test(test)

In [4]:
class Test():
    cls_var = "class variable"
    def __init__(self):
        self.inst_var = "instance variable"
test = Test()
test

In [5]:
help(test)

Help on Test in module __main__ object:

class Test(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  cls_var = 'class variable'



In [6]:
dir(test)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'cls_var',
 'inst_var']

In [1]:
int

int

In [2]:
type(int)

type

In [ ]:
help(in)